# Experiment Lifting

### Load Model & Dataset

In [4]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from src.models.simple_3d_model import LinearModel
import src.bug_dataset
from src.eval.accuracies import keypoint_depth_pck

import torch
from torchvision import transforms
from torch.utils.data import DataLoader

In [5]:
# Plotting Code
%matplotlib widget
import matplotlib.pyplot as plt

def plot_stick_bug(ax, points, vis, prediction=False):
    limb_ranges=[[0,7],[8,14],[15,21],[22,28],[29,35],[36,42],[43,49],[53,56],[59,62]]
    reduced_limb_ranges = [[0,4],[4,7],[7,10],[10,13],[13,16],[16,19],[19,22],[22,25],[25,28]]
    if len(points) == 62:
        points = np.array(points).T
        for num in range(len(limb_ranges)):
            visible_limb = []
            for x in range(limb_ranges[num][0],limb_ranges[num][1]):
                if vis[x]== 1:
                    visible_limb.append(x) 
            if prediction:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb],'--', alpha=0.7, label='Prediction', color='red')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='X',s=10, color='red')
            else:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], label='Actual', color='blue')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='o',s=4, color='blue')
        return ax, line
    elif len(points) == 28:
        points = np.array(points).T
        for num in range(len(reduced_limb_ranges)):
            visible_limb = []
            for x in range(reduced_limb_ranges[num][0],reduced_limb_ranges[num][1]):
                if vis[x]== 1:
                    visible_limb.append(x) 
            if prediction:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb],'--', alpha=0.7, label='Prediction', color='red')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='X',s=10, color='red')
            else:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], label='Actual', color='blue')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='o',s=4, color='blue')
        return ax, line

def set_axes_equal(ax):
    # workaround, as matplotlib's 3D plot has no option for equisised axes (10/2021)
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])


In [6]:
target_dir = "../data/single_sungaya/"
out_df = pd.read_hdf(os.path.join(target_dir, "Data_3D_Pose.hdf5"))
reduceKeypoints = True
sungaya_dataset = src.bug_dataset.BugDataset(df=out_df,
                             root_dir=target_dir, reduced=reduceKeypoints, transform=transforms.Compose([
                                src.bug_dataset.ToTensor()
                                   ]))
train_split = 0.7
valid_split = 0.1
train_size = int(train_split * len(sungaya_dataset))
valid_size = int(valid_split * len(sungaya_dataset))
test_size = len(sungaya_dataset) - (train_size+valid_size)
_,_ ,test_dataset = torch.utils.data.random_split(sungaya_dataset, [train_size, valid_size, test_size], generator=torch.Generator().manual_seed(42))

batch_size = 128
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))   

if reduceKeypoints:
    model = LinearModel(2*28,28,1024) # Linear size 512 looked promising 256 too small
else:
    model = LinearModel(2*62,62,512)

model = model.to(device)
path = "../models/lifting_depth/lifting_depth_model_20220323-150326_150"
model.load_state_dict(torch.load(path))
model.eval()
trained_model = model

Using cuda device


In [13]:
def test_func(model,ds, thr=5.0):
    test_epoch_acc, test_epoch_loss = 0, 0
    test_epoch_acc_kps = [0]*28
    batches = 0
    # Code get the first batch of results
    with torch.no_grad():
        for data in tqdm(ds, desc="Depth Lifting Experiment"):
            image = data['file_name']
            X = data['key_points_2D']
            y = data['key_points_3D'][:,:,2] 
            mask = data['visibility'].to(device)

            X, y = X.to(device, dtype=torch.float), y.to(device, dtype=torch.float)
            
            pred = model(torch.flatten(X, start_dim=1, end_dim=2))
            
            test_loss = torch.mean(((pred - y)*mask)**2) # 
                    
            test_acc,test_acc_kps = keypoint_depth_pck(pred.detach().cpu().numpy(),
                                        y.detach().cpu().numpy(), 
                                        mask.detach().cpu().numpy(),
                                        sungaya_dataset.std_3d[:,2], sungaya_dataset.means_3d[:,2], threshold = thr)
            test_epoch_loss += test_loss.item()
            test_epoch_acc += test_acc
            test_epoch_acc_kps += test_acc_kps
            batches +=1
            
    test_acc = test_epoch_acc/batches
    test_acc_kps = test_epoch_acc_kps/batches
    test_loss = test_epoch_loss/batches
    return test_acc, test_loss, test_acc_kps


In [14]:
trained_model.eval()
test_acc, test_loss, test_acc_kps = test_func(trained_model, test_dataloader)
print(test_acc)
print(test_loss)
print(test_acc_kps)

Depth Lifting Experiment:   0%|          | 0/6 [00:00<?, ?it/s]

In [134]:
# Plots the accuracies per KP at normal threshold
 
fig, ax = plt.subplots()

ax.barh(np.array(label_names)[sungaya_dataset.reduced_kp], test_acc_kps, linewidth =5, edgecolor="white")
ax.invert_yaxis()
# ax.set(xlim=(-1, 8), xticks=np.arange(-1, 8),
#        ylim=(0, 1), yticks=np.arange(0, 0.5))
plt.title("PCK Accuracies for every Keypoint")

plt.xlabel("Accuracies")
plt.ylabel("Keypoints")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# Need to do an experiment where we vary the threshold value on the test set
thresholds = [0.01 ,0.1, 1.0, 2.5, 5.0, 10.0, 15.0, 20.0]
accs = []
losses = []
acc_kps = []
for thr in thresholds:
    test_acc, test_loss, test_acc_kps = test_func(trained_model, test_dataloader, thr)
    accs.append(test_acc)
    losses.append(test_loss)
    acc_kps.append(test_acc_kps)

Depth Lifting Experiment:   0%|          | 0/6 [00:00<?, ?it/s]

Depth Lifting Experiment:   0%|          | 0/6 [00:00<?, ?it/s]

Depth Lifting Experiment:   0%|          | 0/6 [00:00<?, ?it/s]

Depth Lifting Experiment:   0%|          | 0/6 [00:00<?, ?it/s]

Depth Lifting Experiment:   0%|          | 0/6 [00:00<?, ?it/s]

Depth Lifting Experiment:   0%|          | 0/6 [00:00<?, ?it/s]

Depth Lifting Experiment:   0%|          | 0/6 [00:00<?, ?it/s]

Depth Lifting Experiment:   0%|          | 0/6 [00:00<?, ?it/s]

In [102]:
# plots PCK for different thresholds
fig, ax = plt.subplots()

ax.bar(list(map(str,thresholds)), accs, width=1, edgecolor="white", linewidth=5)

# ax.set(xlim=(-1, 8), xticks=np.arange(-1, 8),
#        ylim=(0, 1), yticks=np.arange(0, 0.5))
plt.title("PCK Accuracies at Varying Thresholds")
plt.xlabel("Thresholds")
plt.ylabel("Accuracies")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array(['b_t', 'b_a_2', 'b_a_3', 'b_a_5_end', 'l_1_co_l', 'l_1_ti_l',
       'l_1_pt_l_end', 'l_2_co_l', 'l_2_ti_l', 'l_2_pt_l_end', 'l_3_co_l',
       'l_3_ti_l', 'l_3_pt_l_end', 'l_1_co_r', 'l_1_ti_r', 'l_1_pt_r_end',
       'l_2_co_r', 'l_2_ti_r', 'l_2_pt_r_end', 'l_3_co_r', 'l_3_ti_r',
       'l_3_pt_r_end', 'an_1_r', 'an_3_r', 'an_3_r_end', 'an_1_l',
       'an_3_l', 'an_3_l_end'], dtype='<U12')